In [14]:
import pandas as pd
import numpy as np
from shapely import geometry
from shapely.ops import cascaded_union

In [31]:
#To generate positions to image for tissue slices, select n points at the edge of the desired area for each slice
#and generate n*n tiles centered at the desired region. The tiles that are not overlapped with the desired region are removed. 
#points = pd.read_csv("E:/Meng_temp/191111_CTb_647_sample3/settings/contours.txt", header = None, sep=',')
points = pd.read_csv("E:/Tim/FLARE/191117/positions/picked_positions.txt", header = None, sep=',')

NoOfTissues = 4
NoOfTissuesBottom = 0

In [32]:
a = 20

grid = []
for x in range(-a, a+1):
  for y in range(-a, a+1):
    if x % 2 == 0:
        grid.append((x*200, y*200))
    else:
        grid.append((x*200, -y*200))
    
grid_R =  grid[::-1]
grid = np.array(grid)
grid_R = np.array(grid_R)

In [33]:
n = 7 #number of positions you selected for each slice to define the region of interest

positions = np.empty((0,2))
for i in range(0,NoOfTissuesBottom):
    tissue = geometry.Polygon(points.values[n*i:n*(i+1)])
    center = np.mean(points.values[n*i:n*(i+1)], axis = 0)
    grids = grid_R + center
    saved = np.empty((0,2))
    for j in range(0,len(grids)):
        area = [[grids[j][0]-100,grids[j][1]-100],[grids[j][0]-100,grids[j][1]+100],[grids[j][0]+100,grids[j][1]+100],[grids[j][0]+100,grids[j][1]-100]]
        tile = geometry.Polygon([[p[0], p[1]] for p in area])
        if tissue.intersects(tile) == True:
            saved = np.concatenate((saved,np.array([grids[j]])), axis = 0)
    positions = np.concatenate((positions,saved))
    
for i in range(NoOfTissuesBottom,NoOfTissues):
    tissue = geometry.Polygon(points.values[n*i:n*(i+1)])
    center = np.mean(points.values[n*i:n*(i+1)], axis = 0)
    grids = grid + center
    saved = np.empty((0,2))
    for j in range(0,len(grids)):
        area = [[grids[j][0]-100,grids[j][1]-100],[grids[j][0]-100,grids[j][1]+100],[grids[j][0]+100,grids[j][1]+100],[grids[j][0]+100,grids[j][1]-100]]
        tile = geometry.Polygon([[p[0], p[1]] for p in area])
        if tissue.intersects(tile) == True:
            saved = np.concatenate((saved,np.array([grids[j]])), axis = 0)
    positions = np.concatenate((positions,saved))

In [34]:
#np.savetxt('E:/Meng_temp/191111_CTb_647_sample3/settings/tiles_before.txt', positions, delimiter=',')
np.savetxt('E:/Tim/FLARE/191117/positions/positions.txt', positions, delimiter=',')